# 词向量运算

由于训练词向量计算量非常大，大多数机器学习从业者会加载**预训练词向量**。

**完成本作业后，你将能够：**

- 加载预训练词向量，并使用余弦相似度（cosine similarity）衡量相似性  
- 使用词向量解决类比问题，例如：Man 对应 Woman，King 对应 ______  
- 修改词向量以减少性别偏差  

让我们开始吧！运行以下代码单元以加载所需的包。


In [1]:
# ---------------------------- 导入库 ---------------------------- #

# os: 用于操作系统相关功能，如文件路径、文件夹操作等
import os

# urllib.request: 用于从网络下载文件，例如下载数据集或预训练模型
import urllib.request

# zipfile: 用于解压 zip 文件
import zipfile

# collections: 提供额外的数据结构工具，如 Counter、defaultdict 等
import collections

# numpy: 数值计算库，支持多维数组操作、矩阵运算等
import numpy as np

# torch: PyTorch 核心库，用于张量操作、自动求导和深度学习模型构建
import torch

# torch.nn: PyTorch 的神经网络模块，包含各种神经网络层、损失函数等
import torch.nn as nn

# torch.nn.functional: 提供各种函数形式的神经网络操作，如激活函数、卷积操作等
import torch.nn.functional as F


### 参数设置

In [2]:
# ---------------------------- 超参数设置 ---------------------------- #

# window_size: 上下文窗口大小（Skip-gram 或 CBOW 模型中使用）
# - 意味着每个目标词的上下文词的范围为 window_size
window_size = 3

# vector_dim: 词向量的维度（embedding size）
# - 每个词会被映射为一个 300 维的向量
vector_dim = 300

# epochs: 模型训练轮数
# - 表示整个训练数据将被迭代多少次
epochs = 1000

# ---------------------------- 验证集参数 ---------------------------- #

# valid_size: 从词汇表中随机选择多少个词作为验证样本
valid_size = 16

# valid_window: 从最常出现的前 valid_window 个词中选择验证样本
# - 这里设置为 100，意味着从频率最高的 100 个词中随机挑选
valid_window = 100

# valid_examples: 随机选择 valid_size 个词作为验证样本的索引
# - np.random.choice(a, size, replace=False) 从 0~a-1 中随机选择 size 个不重复的索引
valid_examples = np.random.choice(valid_window, valid_size, replace=False)


### 辅助函数

In [2]:
# ---------------------------- 数据下载与处理 ---------------------------- #

# ----------------------------
# 函数: maybe_download
# ----------------------------
def maybe_download(filename, url, expected_bytes):
    """
    功能：
        检查文件是否存在，如果不存在则从指定 URL 下载，并验证文件大小。
    
    参数：
        filename (str): 文件名（下载后保存的本地名称）
        url (str): 下载 URL 的前缀
        expected_bytes (int): 期望文件大小，用于验证下载是否正确
    
    返回：
        filename (str): 本地文件路径
    """
    # 如果文件不存在，则从网络下载
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    
    # 获取文件信息
    statinfo = os.stat(filename)
    
    # 检查文件大小是否匹配
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        # 文件大小不对，抛出异常
        raise Exception(f'Failed to verify {filename}')
    
    return filename

# ----------------------------
# 函数: read_data
# ----------------------------
def read_data(filename):
    """
    功能：
        读取 zip 文件中的文本数据，并拆分为单词列表。
    
    参数：
        filename (str): 本地 zip 文件路径
    
    返回：
        data (list of str): 文本中的所有单词组成的列表
    """
    # 打开 zip 文件
    with zipfile.ZipFile(filename) as f:
        # 读取 zip 中的第一个文件，并解码为 utf-8
        data = f.read(f.namelist()[0]).decode('utf-8').split()
    
    return data

# ----------------------------
# 函数: build_dataset
# ----------------------------
def build_dataset(words, n_words):
    """
    功能：
        构建词汇表和数据集，将单词映射为索引。
    
    参数：
        words (list of str): 文本中所有单词
        n_words (int): 词汇表大小（只保留最常用的 n_words 个词，其余标记为 'UNK'）
    
    返回：
        data (list of int): 文本单词对应的索引列表
        count (list of [word, count]): 词频统计，首个元素为 'UNK'
        dictionary (dict): {word: index} 词到索引的映射
        reversed_dictionary (dict): {index: word} 索引到词的映射
    """
    # 初始化 count 列表，首个元素为 'UNK'，-1表示尚未计算的词频
    count = [['UNK', -1]]
    
    # 统计词频，并加入到 count，最多 n_words-1 个
    count.extend(collections.Counter(words).most_common(n_words - 1))
    
    # 构建词到索引的映射字典
    dictionary = {word: i for i, (word, _) in enumerate(count)}
    
    # 将文本中的单词转换为索引
    data = []
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)  # 如果不在字典中，索引为 0，对应 'UNK'
        if index == 0:
            unk_count += 1  # 统计未知词数量
        data.append(index)
    
    # 更新 'UNK' 的计数
    count[0][1] = unk_count
    
    # 构建索引到单词的映射
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return data, count, dictionary, reversed_dictionary

# ----------------------------
# 函数: collect_data
# ----------------------------
def collect_data(vocabulary_size=10000):
    """
    功能：
        下载数据集、读取文本、构建词汇表及索引映射。
    
    参数：
        vocabulary_size (int): 词汇表大小，默认为 10000
    
    返回：
        data (list of int): 文本单词对应索引列表
        count (list of [word, count]): 词频统计
        dictionary (dict): 词到索引映射
        reverse_dictionary (dict): 索引到词映射
    """
    # 数据集 URL
    url = 'http://mattmahoney.net/dc/'
    
    # 下载 text8.zip 文件，如果已经存在则直接使用
    filename = maybe_download('text8.zip', url, 31344016)
    
    # 读取文本，得到单词列表
    vocabulary = read_data(filename)
    print(vocabulary[:7])  # 打印前 7 个单词做检查
    
    # 构建数据集：索引列表 + 词频统计 + 字典映射
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)
    
    # 删除原始单词列表，释放内存
    del vocabulary
    
    return data, count, dictionary, reverse_dictionary


In [3]:
# ---------------------------- GloVe 读取 ---------------------------- #

def read_glove_vecs(glove_file):
    """
    功能：
        从 GloVe 文件中读取预训练词向量，将每个单词映射为向量。
    
    参数：
        glove_file (str): GloVe 文件路径（例如 'glove.6B.50d.txt'）
    
    返回：
        words (set): 所有单词的集合
        word_to_vec_map (dict): {word: vector} 映射字典，vector 为 numpy 数组
    """

    # 用于存储所有单词的集合（避免重复）
    words = set()

    # 用于存储单词到向量的映射
    word_to_vec_map = {}

    # 打开 GloVe 文件
    # 每一行格式：word val1 val2 val3 ... valN
    # encoding='utf-8' 避免中文或特殊符号报错
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            # 去除首尾空格并按空格拆分
            line = line.strip().split()

            # 第一个元素是单词
            curr_word = line[0]

            # 将单词加入集合
            words.add(curr_word)

            # 将剩余元素转换为 numpy 数组，作为该单词的向量表示
            # dtype=np.float64 确保浮点精度
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

    return words, word_to_vec_map


接下来，让我们加载词向量。  
在本作业中，我们将使用 **50 维 GloVe 向量**来表示单词。  
运行以下代码单元以加载 `word_to_vec_map`。


In [4]:
# ----------------------------
# 读取 GloVe 词向量
# ----------------------------

# 调用之前定义的 read_glove_vecs 函数
# 参数: 'data/glove.6B.50d.txt' 是 GloVe 预训练词向量文件路径
# 返回两个变量:
#   words: 包含所有单词的集合 (set)，用于快速判断某个单词是否在词向量中
#   word_to_vec_map: 字典 {word: vector}，将每个单词映射到对应的 numpy 向量
words, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

# 打印前 5 个单词及其向量维度作为示例
print("词向量示例:")
for i, word in enumerate(list(words)[:5]):
    print(f"{i+1}. {word}: vector shape = {word_to_vec_map[word].shape}")


词向量示例:
1. wars: vector shape = (50,)
2. bahru: vector shape = (50,)
3. one-to-many: vector shape = (50,)
4. surinaamse: vector shape = (50,)
5. densa: vector shape = (50,)


你已经加载了：

- `words`：词汇表中的单词集合  
- `word_to_vec_map`：将单词映射到其 GloVe 向量表示的字典

你已经看到，独热向量（one-hot vectors）并不能很好地捕捉单词之间的相似性。  
GloVe 向量能够提供关于单词语义的更有用信息。  
现在让我们看看如何使用 GloVe 向量来判断两个单词的相似程度。


# 1 - 余弦相似度（Cosine similarity）

为了衡量两个单词的相似度，我们需要一种方法来测量它们对应词向量之间的相似程度。  
给定两个向量 $u$ 和 $v$，余弦相似度定义如下：

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 \, ||v||_2} = \cos(\theta) \tag{1}$$

其中：
- $u \cdot v$ 表示两个向量的点积（或内积）  
- $||u||_2$ 是向量 $u$ 的范数（或长度）  
- $\theta$ 是 $u$ 和 $v$ 之间的夹角  

该相似度取决于 $u$ 和 $v$ 之间的角度。如果 $u$ 和 $v$ 非常相似，它们的余弦相似度接近 1；如果不相似，余弦相似度会较小。

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center> **图 1**：两个向量夹角的余弦值，用于衡量它们的相似程度</center></caption>

**练习**：实现函数 `cosine_similarity()` 来评估词向量之间的相似度。

**提示**：向量 $u$ 的范数定义为：
$$ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2} $$


In [5]:
# ---------------------------- 相似度计算 ---------------------------- #
def cosine_similarity(u, v):
    """
    功能：
        计算两个向量之间的余弦相似度 (Cosine Similarity)，衡量方向相似性。
        在 NLP 中常用于词向量相似度计算。
    
    参数：
        u (numpy.ndarray): 向量 u，任意维度
        v (numpy.ndarray): 向量 v，维度须与 u 一致
    
    返回：
        similarity (float): 余弦相似度，取值范围 [-1, 1]
            - 1 表示两个向量方向完全相同
            - 0 表示两个向量正交（无相似性）
            - -1 表示两个向量方向完全相反
    """

    # ----------------------------
    # 1. 计算点积 u · v
    # ----------------------------
    dot = np.dot(u, v)
    # np.dot(u, v) → u 向量和 v 向量的内积
    # 点积越大 → 向量方向越相似

    # ----------------------------
    # 2. 计算向量的 L2 范数（长度）
    # ----------------------------
    norm_u = np.linalg.norm(u)  # ||u||
    norm_v = np.linalg.norm(v)  # ||v||

    # ----------------------------
    # 3. 计算余弦相似度
    # cosθ = (u·v) / (||u|| * ||v||)
    # ----------------------------
    return dot / (norm_u * norm_v)


In [6]:
# ----------------------------
# 词向量示例
# ----------------------------

# 从 GloVe 词向量字典中获取单词对应向量
father = word_to_vec_map["father"]       # "father" 的词向量
mother = word_to_vec_map["mother"]       # "mother" 的词向量
ball = word_to_vec_map["ball"]           # "ball" 的词向量
crocodile = word_to_vec_map["crocodile"] # "crocodile" 的词向量
france = word_to_vec_map["france"]       # "france" 的词向量
italy = word_to_vec_map["italy"]         # "italy" 的词向量
paris = word_to_vec_map["paris"]         # "paris" 的词向量
rome = word_to_vec_map["rome"]           # "rome" 的词向量

# ----------------------------
# 1. 计算 father 与 mother 的相似度
# ----------------------------
# 预期结果：相似度较高，因为语义上都是父母相关
print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))

# ----------------------------
# 2. 计算 ball 与 crocodile 的相似度
# ----------------------------
# 预期结果：相似度较低，因为语义上不相关
print("cosine_similarity(ball, crocodile) = ", cosine_similarity(ball, crocodile))

# ----------------------------
# 3. 计算 analogy 示例：france - paris 与 rome - italy
# ----------------------------
# 解释：
#   - "france - paris" 表示从法国减去首都信息
#   - "rome - italy" 表示从罗马减去国家信息
#   - 对比向量差的余弦相似度，用于验证词向量能够捕捉国家-首都关系
print("cosine_similarity(france - paris, rome - italy) = ", 
      cosine_similarity(france - paris, rome - italy))


cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.2743924626137943
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174204


**Expected Output**:

<table>
    <tr>
        <td>
            **cosine_similarity(father, mother)** =
        </td>
        <td>
         0.890903844289
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(ball, crocodile)** =
        </td>
        <td>
         0.274392462614
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(france - paris, rome - italy)** =
        </td>
        <td>
         -0.675147930817
        </td>
    </tr>
</table>

在得到正确的预期输出后，你可以自由修改输入，测量其他单词对之间的余弦相似度！  
尝试不同输入的余弦相似度可以帮助你更好地理解词向量的行为。


## 2 - 词类比任务（Word analogy task）

在词类比任务中，我们要完成句子：<font color='brown'>"*a* is to *b* as *c* is to **____**"</font>。  
一个例子是：<font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>。  

具体来说，我们试图找到一个单词 *d*，使得对应的词向量 $e_a, e_b, e_c, e_d$ 满足关系：

$$e_b - e_a \approx e_d - e_c$$

我们将使用余弦相似度来衡量 $e_b - e_a$ 与 $e_d - e_c$ 之间的相似性。

**练习**：完成下面的代码，实现词类比任务！


In [7]:
# ----------------------------
# 类比问题求解函数
# ----------------------------
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    功能：
        给定三个单词 word_a, word_b, word_c，找到最符合类比关系 "a:b :: c:?" 的单词。
        例如: "man:king :: woman:?" → 答案应该是 "queen"
    
    参数：
        word_a (str): 单词 a
        word_b (str): 单词 b
        word_c (str): 单词 c
        word_to_vec_map (dict): 词向量字典 {word: vector}
    
    返回：
        best_word (str): 最符合类比关系的单词
    """

    # ----------------------------
    # 1. 转为小写，保证大小写不影响查找
    # ----------------------------
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()

    # ----------------------------
    # 2. 获取三个单词对应的词向量
    # ----------------------------
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]

    # ----------------------------
    # 3. 遍历所有单词，寻找最匹配的类比
    # ----------------------------
    words = word_to_vec_map.keys()  # 获取所有单词
    max_cosine_sim = -100           # 初始化最大余弦相似度为很小的值
    best_word = None                # 初始化最佳单词为空

    for word in words:
        # 跳过输入的三个单词，避免自己匹配自己
        if word in [word_a, word_b, word_c]:
            continue

        # 计算余弦相似度
        # 类比公式: vec_b - vec_a ≈ vec_best - vec_c → vec_best ≈ vec_c + (vec_b - vec_a)
        # 实际计算余弦相似度: cos((b - a), (word - c))
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[word] - e_c)

        # 如果相似度更大，则更新最佳单词
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = word

    # 返回找到的最佳单词
    return best_word


运行下方代码单元以测试你的代码，这可能需要 1-2 分钟。


In [8]:
# ----------------------------
# 类比实验：测试多个三元组
# ----------------------------

# triads_to_try: 待测试的三元组列表，每个三元组格式 (word_a, word_b, word_c)
# 意思是求 "a : b :: c : ?" 的类比结果
triads_to_try = [
    ('italy', 'italian', 'spain'),   # 意大利:意大利语 :: 西班牙: ?
    ('india', 'delhi', 'japan'),     # 印度:德里 :: 日本: ?
    ('man', 'woman', 'boy'),         # 男人:女人 :: 男孩: ?
    ('small', 'smaller', 'large')    # 小:更小 :: 大: ?
]

# 遍历每个三元组，调用 complete_analogy() 找出最符合类比的单词
for triad in triads_to_try:
    # *triad 将三元组拆开作为函数参数传入
    result_word = complete_analogy(*triad, word_to_vec_map)

    # 格式化输出：a -> b :: c -> 预测结果
    print('{} -> {} :: {} -> {}'.format(*triad, result_word))


italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger


**Expected Output**:

<table>
    <tr>
        <td>
            **italy -> italian** ::
        </td>
        <td>
         spain -> spanish
        </td>
    </tr>
        <tr>
        <td>
            **india -> delhi** ::
        </td>
        <td>
         japan -> tokyo
        </td>
    </tr>
        <tr>
        <td>
            **man -> woman ** ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            **small -> smaller ** ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

在得到正确的预期输出后，你可以自由修改上方的输入单元，测试你自己的类比题。  
尝试找到一些算法能够正确解决的类比对，也尝试一些算法无法给出正确答案的类比，例如：small -> smaller as big -> ?。


### 恭喜！

你已经完成了本次作业。以下是你需要记住的主要内容：

- **余弦相似度**是比较词向量对相似性的一种有效方法。（虽然 L2 距离也可用）  
- 对于 NLP 应用，从网上获取预训练的词向量通常是一个很好的起点。



## 3 - 消除词向量偏差


在接下来的练习中，你将观察词向量中可能反映出的性别偏差，并探索减少偏差的算法。  
除了学习去偏置（debiasing）相关知识外，这个练习还能帮助你加深对词向量行为的直观理解。  

本部分涉及一些线性代数知识，但即使你不是线性代数专家，也可以完成练习，我们鼓励你尝试。

首先，让我们看看 GloVe 词向量是如何与性别相关的。  
你将首先计算向量：

$$g = e_{woman} - e_{man}$$

其中，$e_{woman}$ 表示单词 *woman* 的词向量，$e_{man}$ 表示单词 *man* 的词向量。  
得到的向量 $g$ 大致编码了“性别”这一概念。（如果你计算 $g_1 = e_{mother}-e_{father}$、$g_2 = e_{girl}-e_{boy}$ 等并取平均，可能会得到更准确的表示，但仅使用 $e_{woman}-e_{man}$ 对当前练习来说已经足够。）


In [9]:
# ----------------------------
# 计算词向量差
# ----------------------------

# 计算向量差 g = vec(woman) - vec(man)
# 这个向量 g 捕捉了 “从 man 到 woman 的语义方向”
g = word_to_vec_map['woman'] - word_to_vec_map['man']

# 打印向量 g
# 输出是一个 numpy 数组，长度等于词向量维度（例如 50、100、300）
# 每个元素表示在该维度上 woman 相对于 man 的向量差
print(g)


[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


现在，你将考虑不同单词与向量 $g$ 的余弦相似度。  
请思考余弦相似度为正值意味着什么，以及为负值又意味着什么。


In [11]:
# ----------------------------
# 输出与构建向量 g 相似度的名字列表
# ----------------------------

print('List of names and their similarities with constructed vector:')

# name_list: 待比较的名字列表，可以包含男性、女性或其他名字
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

# 遍历每个名字，计算其词向量与向量 g 的余弦相似度
for w in name_list:
    # word_to_vec_map[w]: 获取名字 w 的词向量
    # g: 之前计算的向量差 g = vec('woman') - vec('man')
    # cosine_similarity(u, v): 计算两个向量的余弦相似度
    sim = cosine_similarity(word_to_vec_map[w], g)
    
    # 打印名字和相似度
    # 余弦相似度越接近 1，说明该名字的词向量方向与 g 越接近
    print(w, sim)


List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.315597935396073
sophie 0.31868789859418784
ronaldo -0.3124479685032943
priya 0.17632041839009402
rahul -0.1691547103923172
danielle 0.24393299216283895
reza -0.07930429672199552
katy 0.2831068659572615
yasmin 0.23313857767928758


如你所见，女性名字的余弦相似度通常与我们构建的向量 $g$ 为正，而男性名字通常为负。  
这并不令人惊讶，结果看起来也可以接受。

接下来，让我们尝试一些其他单词。


In [10]:
# ----------------------------
# 输出其他单词与向量 g 的相似度
# ----------------------------

print('Other words and their similarities:')

# word_list: 待比较的词列表，可以包含职业、物品、概念等
word_list = [
    'lipstick', 'guns', 'science', 'arts', 'literature', 'warrior',
    'doctor', 'tree', 'receptionist', 'technology', 'fashion', 
    'teacher', 'engineer', 'pilot', 'computer', 'singer'
]

# 遍历每个单词，计算其词向量与向量 g 的余弦相似度
for w in word_list:
    # word_to_vec_map[w]: 获取单词 w 的词向量
    # g: 之前计算的向量差 g = vec('woman') - vec('man')
    # cosine_similarity(u, v): 计算两个向量的余弦相似度
    sim = cosine_similarity(word_to_vec_map[w], g)
    
    # 打印单词和相似度
    # 余弦相似度越接近 1，说明该词向量方向与 g 越接近
    print(w, sim)


Other words and their similarities:
lipstick 0.27691916256382665
guns -0.1888485567898898
science -0.060829065409296994
arts 0.008189312385880328
literature 0.06472504433459927
warrior -0.20920164641125288
doctor 0.11895289410935041
tree -0.07089399175478091
receptionist 0.33077941750593737
technology -0.131937324475543
fashion 0.03563894625772699
teacher 0.17920923431825664
engineer -0.08039280494524072
pilot 0.001076449899191679
computer -0.10330358873850498
singer 0.1850051813649629


你注意到什么令人惊讶的现象了吗？这些结果惊人地反映了某些不健康的性别刻板印象。例如，“computer”更接近“man”，而“literature”更接近“woman”。哎呀！

下面我们将展示如何使用 [Bolukbasi et al., 2016](https://arxiv.org/abs/1607.06520) 的算法来减少这些向量的偏差。  
注意，有些词对（如 "actor"/"actress" 或 "grandmother"/"grandfather"）应保持性别特定，而其他词（如 "receptionist" 或 "technology"）应被中性化，即不再与性别相关。去偏置时，需要对这两类词采取不同的处理方式。

### 3.1 - 对非性别特定词进行中性化

下图有助于你理解中性化操作的效果。  
如果你使用的是 50 维词向量，50 维空间可以分为两部分：  
- 偏置方向 $g$  
- 剩余 49 维，我们称之为 $g_{\perp}$  

在线性代数中，$g_{\perp}$ 与 $g$ 垂直（或“正交”），即两者夹角为 90 度。  
中性化步骤将诸如 $e_{receptionist}$ 的向量在 $g$ 方向上的分量置零，从而得到 $e_{receptionist}^{debiased}$。

虽然 $g_{\perp}$ 是 49 维的，但由于屏幕绘图的限制，我们使用 1 维轴来示意。

<img src="images/neutral.png" style="width:800px;height:300px;">
<caption><center> **图 2**：“receptionist”的词向量，在应用中性化操作前后表示。</center></caption>

**练习**：实现 `neutralize()`，去除诸如 "receptionist" 或 "scientist" 等词的偏差。  
给定输入嵌入 $e$，你可以使用以下公式计算 $e^{debiased}$：

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

如果你是线性代数专家，你可能会认出 $e^{bias\_component}$ 是 $e$ 在 $g$ 方向上的投影。  
如果不是专家，也无需担心。


In [13]:
# ----------------------------
# 词向量偏差消除（Neutralize）
# ----------------------------

def neutralize(word, g, word_to_vec_map):
    """
    功能：
        对单词的词向量进行性别偏差消除（neutralize）。
        将词向量中沿着性别方向 g 的分量去除，使其与性别无关。
    
    参数：
        word (str): 待处理的单词
        g (np.array): 性别方向向量（通常 g = vec('woman') - vec('man')）
        word_to_vec_map (dict): 单词到词向量的映射字典
    
    返回：
        e_debiased (np.array): 去除性别偏差后的词向量
    """

    # ----------------------------
    # 1. 获取单词的原始词向量
    # ----------------------------
    e = word_to_vec_map[word]
    # e: shape = (vector_dim,)，例如 (300,)

    # ----------------------------
    # 2. 计算沿性别方向 g 的分量
    # ----------------------------
    # np.dot(e, g) -> 计算 e 在 g 方向的投影长度
    # np.linalg.norm(g)**2 -> g 的平方模，用于归一化
    # * g -> 投影向量，即 e 在 g 方向上的成分
    e_biascomponent = np.dot(e, g) / np.square(np.linalg.norm(g)) * g

    # ----------------------------
    # 3. 去除偏差分量
    # ----------------------------
    # 原始词向量减去性别方向上的分量
    e_debiased = e - e_biascomponent

    # 返回去偏后的词向量
    return e_debiased


In [14]:
# ----------------------------
# 性别偏差消除示例
# ----------------------------

# 定义待处理的单词
e = "receptionist"

# ----------------------------
# 1. 计算并打印去偏前的词向量与性别方向 g 的相似度
# ----------------------------
# word_to_vec_map["receptionist"]: 获取 "receptionist" 的原始词向量
# cosine_similarity(..., g): 计算词向量与性别方向向量 g 的余弦相似度
print("cosine similarity between " + e + " and g, before neutralizing: ", 
      cosine_similarity(word_to_vec_map["receptionist"], g))
# 余弦相似度越接近 1，说明原词向量沿性别方向偏差越明显

# ----------------------------
# 2. 对词向量进行性别偏差消除
# ----------------------------
# 调用 neutralize 函数，将词向量中沿性别方向 g 的分量去除
e_debiased = neutralize("receptionist", g, word_to_vec_map)

# ----------------------------
# 3. 计算并打印去偏后的词向量与性别方向 g 的相似度
# ----------------------------
# 此时余弦相似度应接近 0，说明性别方向成分已被去除
print("cosine similarity between " + e + " and g, after neutralizing: ", 
      cosine_similarity(e_debiased, g))


cosine similarity between receptionist and g, before neutralizing:  0.33077941750593737
cosine similarity between receptionist and g, after neutralizing:  1.1682064664487028e-17


**Expected Output**: The second result is essentially 0, up to numerical roundof (on the order of $10^{-17}$).


<table>
    <tr>
        <td>
            **cosine similarity between receptionist and g, before neutralizing:** :
        </td>
        <td>
         0.330779417506
        </td>
    </tr>
        <tr>
        <td>
            **cosine similarity between receptionist and g, after neutralizing:** :
        </td>
        <td>
         -3.26732746085e-17
    </tr>
</table>

### 3.2 - 针对性别特定词的均衡算法（Equalization）

接下来，让我们看看去偏置如何应用于诸如 "actress" 和 "actor" 这类词对。  
均衡算法应用于那些你希望仅在性别属性上有所差异的词对。  

具体例子：假设 "actress" 比 "actor" 更接近 "babysit"。通过对 "babysit" 进行中性化操作，可以减少与性别刻板印象相关的偏差。但这仍无法保证 "actor" 和 "actress" 到 "babysit" 的距离相等。均衡算法可以解决这一问题。

均衡算法的关键思想是确保特定词对在 49 维 $g_\perp$ 空间中等距分布。  
均衡步骤还确保两个经过均衡的词与 $e_{receptionist}^{debiased}$ 或其他已中性化词保持相同距离。  

下图展示了均衡操作的原理：

<img src="images/equalize10.png" style="width:800px;height:400px;">

线性代数的推导稍微复杂一些。（详细内容请参见 Bolukbasi et al., 2016。）  
关键方程如下：

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$
\mu_B = \frac{\mu \cdot \text{bias\_axis}}{||\text{bias\_axis}||_2^2} \cdot \text{bias\_axis} \tag{5}
$$


$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$e_{w1B} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{(e_{\text{w1}} - \mu_{\perp}) - \mu_B} {|(e_{w1} - \mu_{\perp}) - \mu_B)|} \tag{7}$$

$$e_{w2B} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{(e_{\text{w2}} - \mu_{\perp}) - \mu_B} {|(e_{w2} - \mu_{\perp}) - \mu_B)|} \tag{8}$$

$$e_{w1B}^{corrected} = \sqrt{ |1 - ||\mu_{\perp} ||^2_2| } * \frac{e_{w1B} - \mu_B}{|(e_{w1} - \mu_{\perp}) - \mu_B|} \tag{9}$$

$$e_{w2B}^{corrected} = \sqrt{ |1 - ||\mu_{\perp} ||^2_2| } * \frac{e_{w2B} - \mu_B}{|(e_{w2} - \mu_{\perp}) - \mu_B|} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$

**练习**：实现下面的函数，使用以上公式得到词对的最终均衡版本。祝你好运！


In [15]:
# ----------------------------
# 词向量对齐（Equalize）函数
# ----------------------------
def equalize(pair, bias_axis, word_to_vec_map):
    """
    功能：
        对一对词（如 'actor' vs 'actress'）进行性别偏差校正，使它们在性别方向上对称，
        并保持它们在性别方向正交部分的平均值不变。
    
    参数：
        pair (tuple of str): 待处理的词对，例如 ('actor', 'actress')
        bias_axis (np.array): 性别方向向量 g
        word_to_vec_map (dict): 词到词向量的映射
        
    返回：
        e1, e2 (np.array): 校正后的两词向量
    """

    # ----------------------------
    # 1. 获取词对的原始向量
    # ----------------------------
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]

    # ----------------------------
    # 2. 计算两向量的均值向量
    # ----------------------------
    mu = (e_w1 + e_w2) / 2.0  # 平均向量

    # ----------------------------
    # 3. 计算均值向量在性别方向上的投影
    # ----------------------------
    # mu_B 是 mu 在 bias_axis 上的分量
    mu_B = np.dot(mu, bias_axis) / np.square(np.linalg.norm(bias_axis)) * bias_axis

    # ----------------------------
    # 4. 计算均值向量在性别方向正交部分
    # ----------------------------
    # mu_orth 是 mu 在性别方向上的正交分量
    mu_orth = mu - mu_B

    # ----------------------------
    # 5. 计算每个词向量在性别方向上的投影
    # ----------------------------
    e_w1B = np.dot(e_w1, bias_axis) / np.square(np.linalg.norm(bias_axis)) * bias_axis
    e_w2B = np.dot(e_w2, bias_axis) / np.square(np.linalg.norm(bias_axis)) * bias_axis

    # ----------------------------
    # 6. 计算校正后的性别方向分量
    # ----------------------------
    # 公式中保证校正后的向量在性别方向上对称，并保留在正交方向的均值
    corrected_e_w1B = np.sqrt(abs(1 - np.square(np.linalg.norm(mu_orth)))) * (e_w1B - mu_B) / abs(e_w1 - mu_orth - mu_B)
    corrected_e_w2B = np.sqrt(abs(1 - np.square(np.linalg.norm(mu_orth)))) * (e_w2B - mu_B) / abs(e_w2 - mu_orth - mu_B)

    # ----------------------------
    # 7. 将校正后的性别分量加回正交部分
    # ----------------------------
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth

    # 返回校正后的向量
    return e1, e2


In [16]:
# ----------------------------
# 测试 equalize 函数
# ----------------------------

print("cosine similarities before equalizing:")
# 输出校正前 "man" 和 "woman" 与性别向量 g 的余弦相似度
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", 
      cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", 
      cosine_similarity(word_to_vec_map["woman"], g))
print()  # 空行用于美观

# ----------------------------
# 对 ('man', 'woman') 进行 equalize 校正
# ----------------------------
# 调用 equalize 函数，返回校正后的词向量 e1 和 e2
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)

print("cosine similarities after equalizing:")
# 输出校正后词向量与性别向量 g 的余弦相似度
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))


cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.1171109576533683
cosine_similarity(word_to_vec_map["woman"], gender) =  0.3566661884627037

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7165727525843935
cosine_similarity(e2, gender) =  0.7396596474928908


**Expected Output**:

cosine similarities before equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(word_to_vec_map["man"], gender)** =
        </td>
        <td>
         -0.117110957653
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(word_to_vec_map["woman"], gender)** =
        </td>
        <td>
         0.356666188463
        </td>
    </tr>
</table>

cosine similarities after equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(u1, gender)** =
        </td>
        <td>
         -0.700436428931
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(u2, gender)** =
        </td>
        <td>
         0.700436428931
        </td>
    </tr>
</table>

请随意修改上面单元格中的输入词，将均衡算法应用于其他词对。

这些去偏置算法在减少偏见方面非常有帮助，但并不完美，不能完全消除所有偏见痕迹。  
例如，本实现的一大缺点是偏置方向 $g$ 仅使用了 _woman_ 和 _man_ 这对词来定义。正如之前讨论的，如果通过计算 $g_1 = e_{woman} - e_{man}$；$g_2 = e_{mother} - e_{father}$；$g_3 = e_{girl} - e_{boy}$ 等并对它们取平均，你将得到更准确的“性别”维度估计，用于 50 维的词向量空间。  
你也可以尝试这种变体来进行实验。


### 恭喜

你已经完成了本笔记本，并了解了词向量的多种应用方式以及如何对其进行修改。

恭喜你完成本笔记本的学习！


**参考文献**：
- 去偏置算法来自 Bolukbasi 等人, 2016，[Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- GloVe 词向量由 Jeffrey Pennington, Richard Socher 和 Christopher D. Manning 提出。(https://nlp.stanford.edu/projects/glove/)
